In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [3]:
##loading the dataset
data=pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
#Preprocessing
#dropping irrelevant features
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
7401,633,France,Female,39,9,129189.15,2,0,0,170998.83,0
9091,745,France,Female,40,6,131184.67,1,1,1,49815.62,0
9198,499,France,Female,28,4,141792.61,1,1,1,22001.91,0
8085,794,France,Male,33,0,0.00,2,0,0,178122.71,0
7426,619,France,Female,48,4,0.00,1,0,0,18094.96,1


In [5]:
##Encoding categorical variables
le_gender=LabelEncoder()
data["Gender"]=le_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [6]:
##OnehotEncoding Geography column
from sklearn.preprocessing import OneHotEncoder
ohe_geo=OneHotEncoder()
geo_encoder=ohe_geo.fit_transform(data[["Geography"]])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
ohe_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [8]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], shape=(10000, 3))

In [9]:
geo_encoded_data=pd.DataFrame(geo_encoder.toarray(),columns=ohe_geo.get_feature_names_out(["Geography"]))
geo_encoded_data.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [10]:
#combining encoded and original data
data=pd.concat([data.drop(["Geography"],axis=1),geo_encoded_data],axis=1)

In [11]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
##Saving the encoders and scalers
with open('le_gender.pkl','wb') as file:
    pickle.dump(le_gender,file)

with open('ohe_geo.pkl','wb') as file:
    pickle.dump(ohe_geo,file)

In [13]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [14]:
###dividing dataset into dependent and independent features
X=data.drop(["Exited"],axis=1)
y=data["Exited"]

In [15]:
#train and test sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
#scaling
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [17]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN Training

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard


In [19]:
#Building the ANN model
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), #HL! connnected with input layer
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid') #output layer
])

d:\Good Stuff\DL\venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [20]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
##compiling the model
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
#loss=tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [22]:
#setting up tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    update_freq='batch'
)


In [23]:
#setting up early stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [24]:
#training the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)


Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.8369 - loss: 0.3929 - val_accuracy: 0.8505 - val_loss: 0.3630
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8540 - loss: 0.3547 - val_accuracy: 0.8605 - val_loss: 0.3460
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8554 - loss: 0.3514 - val_accuracy: 0.8595 - val_loss: 0.3479
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8612 - loss: 0.3429 - val_accuracy: 0.8555 - val_loss: 0.3513
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8593 - loss: 0.3434 - val_accuracy: 0.8565 - val_loss: 0.3506
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8602 - loss: 0.3410 - val_accuracy: 0.8515 - val_loss: 0.3485
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8599 - loss: 0.3361 - val_accuracy: 0.8580 - val_loss: 0.3449
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8643 - loss: 0.3352 - val_accu

In [25]:
model.save('model.h5')

In [26]:
%load_ext tensorboard

In [27]:
%tensorboard --logdir logs/fit/20260128-193110

Reusing TensorBoard on port 6006 (pid 21812), started 4:14:22 ago. (Use '!kill 21812' to kill it.)

In [28]:
#loading the trained model , scaler pickle, onehot
from tensorflow.keras.models import load_model

model=load_model('model.h5')

#loading the encoders and scalers
with open ("ohe_geo.pkl",'rb') as file:
    encoder_geo=pickle.load(file)

with open ("le_gender.pkl",'rb') as file:
    encoder_gender=pickle.load(file)

with open ("scaler.pkl",'rb') as file:
    scaler=pickle.load(file)

In [29]:
input_data={
    'CreditScore':600,
    'Geography':'France',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [30]:
geo_encoded=encoder_geo.transform([[input_data['Geography']]]).toarray()
geo_encoded=pd.DataFrame(geo_encoded,columns=encoder_geo.get_feature_names_out(['Geography']))
geo_encoded

d:\Good Stuff\DL\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [31]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [32]:
input_df=pd.concat([input_df.reset_index(drop=True),geo_encoded],axis=1)

In [33]:
input_df['Gender']=encoder_gender.transform(input_df['Gender'])

In [34]:
input_df=input_df.drop(['Geography'],axis=1)

In [35]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [36]:
prediction=model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step


array([[0.04723744]], dtype=float32)

In [37]:
prediction_proba=prediction[0][0]
prediction_proba

np.float32(0.047237445)

In [38]:
if prediction_proba>0.5:
    print("customer likely to churn")
else:
    print("customer unlikely to churn")

customer unlikely to churn
